In [26]:
import numpy as np
import scipy.stats as stats

vol = np.array([0.1, 0.2, 0.3])
rho12 = 0.4 
rho13 = 0.5
rho23 = 0.6

def pos_def(m): return np.linalg.det(m) > 0

mcorr = np.array(
    [
        [1,     rho12, rho13],
        [rho12, 1,     rho23],
        [rho13, rho23, 1]
    ]
)
assert pos_def(mcorr), 'Not pos. definite.'
mcov = np.diag(vol) @ mcorr @ np.diag(vol)


distP = stats.multivariate_normal(cov=mcov)

volQ = vol*1.1 # slightly higher implied than realized.

n_cov = 100

nsimP = 100
nsimQ = 1000

T = 3/12


rstate0 = np.random.RandomState(0)
rstate1 = np.random.RandomState(0)


statprice = np.empty((nsimQ, 2))
price = np.empty((nsimP, nsimQ, 2))
for i in range(nsimP):
    XP = distP.rvs(n_cov)
    #mcovP_est = np.cov(X_past, rowvar=Fals, ddof=1)
    mcorrP_est = np.corrcoef(XP, rowvar=False)
    
    mcovQ_est = np.diag(volQ) @ mcorrP_est @ np.diag(volQ)
    
    mcovQ = np.diag(volQ) @ mcorr @ np.diag(volQ)
    
    XQ0 = stats.multivariate_normal(cov=mcovQ)
    XQ = stats.multivariate_normal(cov=mcovQ_est)
    
    for ipath in range(nsimQ):
        ST0 = np.exp((0 - volQ**2/2)*T + volQ*np.sqrt(T)* XQ0.rvs(1, random_state=rstate0))
        ST = np.exp((0 - volQ**2/2)*T + volQ*np.sqrt(T)* XQ.rvs(1, random_state=rstate1))
        
        statprice[ipath, 0] = np.max(ST0)
        statprice[ipath, 1] = np.mean(ST0)
        price[i, ipath, 0] = np.max(ST)
        price[i, ipath, 1] = np.mean(ST)
    

truePrice0 = np.mean(statprice[:,0]) 
truePrice1 = np.mean(statprice[:,1]) 
estPrice0 = np.mean(price[:,:,0])
estPrice1 = np.mean(price[:,:,1])
print((estPrice0/truePrice0 -1 ))
print((estPrice1/truePrice1 -1 ))
    
    


0.0007116134309215205
0.0008456267844161314
